In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import average_precision_score

In [2]:
#загрузка датасетов
b_games = pd.read_feather('bgg_boardgames_top_2000.feather')
ratings = pd.read_feather('bgg_ratings_top_2000.feather')

===== Board Games =====
====================

boardgame_id - board game id <br>
title - title of board game <br>
year_published - year of publication of the board game <br>
minplayers - minimum number of players per game <br>
maxplayers - maximum number of players per game <br>
minplaytime - minimum playing time per game <br>
maxplaytime - maximum playing time per game <br>
age - lower age limit for playing <br>
users_rated - number of users who rated a game <br>
average_rating - average rating <br>
bayes_average_rating - bayes average rating <br>
median - median rating <br>
stddev - standard deviation of rating <br>
owned - number of users who have a game <br>
trading - number of users who selling a game <br>
wishing - number of users who want to get a game <br>
num_of_comments - number of comments <br>
num_of_weights - number of scores for weight <br>
average_weight - average weight of game <br>
ranks - game ranks <br>
main_publisher - main publisher <br>
description - description of the game <br>
publishers - all publishers <br>
honors - all honors <br>
expansions - all expansions <br>
accessories - all accessories <br>
artists - all artists <br>
mechanics - used mechanics <br>
category - category ща пфьуы <br>
designers - all designers <br>
graphic_designers - all graphic designers <br>
subdomains - subdomains of categories <br>
implementations - all implementations <br>
suggested_numplayers - proposed number of players <br>
podcast_episodes - all podcast episodes <br>
comments - some comments <br>
marketplace_history - marketplace history <br>
thumbnail_link - thumbnail link <br>
image_link - image link <br>

===== Ratings =====
================

nickname - user's nickname <br>
title - title of the game <br>
boardgame_id - id of the board game <br>
rating - given rating <br>
num_of_plays - number of games played by the user <br>
comment - comment fot rating <br>
own - flag whether the user owns the game <br>
prevowned - flag of whether the user has previously owned the game <br>
fortrade - flag whether the user is selling the game <br>
want - flag whether the user want the game <br>
wanttoplay - flag whether the user want to play the game <br>
wanttobuy - flag whether the user want to buy the game <br>
wishlist - flag whether the user have the game in wishlist <br>
preordered - flag whether the user preordered the game <br>
last_modified - date of last modification <br>

In [3]:
b_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   boardgame_id          1999 non-null   int32  
 1   title                 1999 non-null   object 
 2   year_published        1999 non-null   Int16  
 3   minplayers            1999 non-null   Int8   
 4   maxplayers            1999 non-null   Int16  
 5   minplaytime           1999 non-null   Int32  
 6   maxplaytime           1999 non-null   Int32  
 7   age                   1999 non-null   Int16  
 8   users_rated           1999 non-null   Int32  
 9   average_rating        1999 non-null   float32
 10  bayes_average_rating  1999 non-null   float32
 11  median                1999 non-null   float32
 12  stddev                1999 non-null   float32
 13  owned                 1999 non-null   Int32  
 14  trading               1999 non-null   Int16  
 15  wishing              

In [4]:
#удаляем столбец, где слишком мало значений
b_games.drop('graphic_designers', axis = 1, inplace=True)

In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003471 entries, 0 to 1003470
Data columns (total 15 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   nickname       1003471 non-null  object 
 1   title          1003471 non-null  object 
 2   boardgame_id   1003471 non-null  int64  
 3   rating         1003402 non-null  float64
 4   num_of_plays   1003471 non-null  int64  
 5   comment        263604 non-null   object 
 6   own            1003471 non-null  int64  
 7   prevowned      1003471 non-null  int64  
 8   fortrade       1003471 non-null  int64  
 9   want           1003471 non-null  int64  
 10  wanttoplay     1003471 non-null  int64  
 11  wanttobuy      1003471 non-null  int64  
 12  wishlist       1003471 non-null  int64  
 13  preordered     1003471 non-null  int64  
 14  last_modified  1003471 non-null  object 
dtypes: float64(1), int64(10), object(4)
memory usage: 114.8+ MB


In [6]:
b_games.head()

,boardgame_id,title,year_published,minplayers,maxplayers,minplaytime,maxplaytime,age,users_rated,average_rating,...,category,designers,subdomains,implementations,suggested_numplayers,podcast_episodes,comments,marketplace_history,thumbnail_link,image_link
0,77423,The Lord of the Rings: The Card Game,2011,1,2,30,60,13,23231,7.66006,...,Adventure|Card Game|Collectible Components|Fan...,Nate French,Customizable Games,The Lord of the Rings: The Card Game – Revised...,1||Best~221|Recommended~368|Not Recommended~45...,005 – Emergent Narrative &amp; Storytelling in...,"0 1 1 2 3 5 8|Recommended by Joel D., Chris S....",unix_saledate|1301347058||saletime|120837||pri...,https://cf.geekdo-images.com/ZmXi5BbtoWl58ZnKc...,https://cf.geekdo-images.com/ZmXi5BbtoWl58ZnKc...
1,88,Torres,1999,2,4,60,60,12,9398,7.09677,...,Abstract Strategy|Medieval,Michael Kiesling|Wolfgang Kramer,Abstract Games|Strategy Games,Torres Family|Burgenland,1||Best~0|Recommended~0|Not Recommended~71|||2...,"BGTG 115 - Spiel des Jahres, Then &amp; Now|BG...",-Johnny-|One play. Didn't do it for me at all....,unix_saledate|1047560215||saletime|9025580||pr...,https://cf.geekdo-images.com/3pL0cEnaGn7Qwn6F2...,https://cf.geekdo-images.com/3pL0cEnaGn7Qwn6F2...
2,203420,Exit: The Game – The Abandoned Cabin,2016,1,6,60,120,12,10691,7.45508,...,Deduction|Puzzle|Real-time,Inka Brand|Markus Brand,Family Games|Thematic Games,None,1||Best~6|Recommended~79|Not Recommended~38|||...,"34. Devon Dice Podcast, Near &amp; Far, Dice F...",-Johnny-|Not actually a game. More of an activ...,unix_saledate|1488266932||saletime|992349||pri...,https://cf.geekdo-images.com/r7cZVZMpP28w3-JQ-...,https://cf.geekdo-images.com/r7cZVZMpP28w3-JQ-...
3,310448,Zombie Teenz Evolution,2020,2,4,15,25,8,1321,7.78381,...,Children's Game|Zombies,Annick Lobet,Family Games,Zombie Kidz Evolution,1||Best~0|Recommended~4|Not Recommended~8|||2|...,"Episode 112 - Bonfire, Hostage Negotiator, Aeo...","21kellie08|Jackson’s||Adam78|I went into this,...",unix_saledate|1608485820||saletime|1710444||pr...,https://cf.geekdo-images.com/elS-EEtVF92Vk-YBq...,https://cf.geekdo-images.com/elS-EEtVF92Vk-YBq...
4,25417,BattleLore,2006,2,2,60,60,10,9264,7.35379,...,Fantasy|Medieval|Miniatures|Wargame,Richard Borg,Thematic Games|Wargames,BattleLore: Second Edition|Battles of Westeros,1||Best~2|Recommended~10|Not Recommended~90|||...,BGWS 035 – BattleLore|D6G Episode 13: Battlelo...,1000rpm|Rating based on 3 plays of the most ba...,unix_saledate|1165637847||saletime|445093||pri...,https://cf.geekdo-images.com/cAH5iJ9gQ7aAj4iwU...,https://cf.geekdo-images.com/cAH5iJ9gQ7aAj4iwU...


In [7]:
ratings.head()

,nickname,title,boardgame_id,rating,num_of_plays,comment,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,last_modified
0,happyjosiah,5-Minute Dungeon,207830,7.0,2,"Like it says on the label, it's a dungeon-craw...",0,0,0,0,0,0,0,0,2019-03-11 13:02:09
1,happyjosiah,6 nimmt!,432,7.0,8,A really solid addition to any game collection...,0,0,0,0,0,0,0,0,2019-10-04 11:16:41
2,happyjosiah,7 Wonders,68448,8.0,41,"While ostensibly a game of card drafting, 7 Wo...",1,0,0,0,0,0,0,0,2012-12-31 10:47:48
3,happyjosiah,7 Wonders Duel,173346,5.0,1,"Rated after one play, but was pretty disappoin...",0,0,0,0,0,0,0,0,2018-03-19 12:58:55
4,happyjosiah,Acquire,5,9.0,11,With a theme of corporate America and a bland-...,1,0,0,0,0,0,0,0,2020-07-22 12:35:58


In [8]:
#выделяем вещественные и категориальные фичи
numeric_columns = b_games.select_dtypes(exclude='object').columns
categorical_columns = b_games.select_dtypes(include='object').columns

In [9]:
#заменим пропуски в вещественных признаках медианой, а в категориальных - самым популярным классом
for col in numeric_columns:
    b_games[col] = b_games[col].fillna(b_games[col].median())
for col in categorical_columns:
    b_games[col] = b_games[col].fillna(b_games[col].mode().iloc[0]).astype('category')

In [10]:
#эта функция проверяет корреляцию признаков
def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

In [11]:
print("Top Absolute Correlations")
print(round(get_top_abs_correlations(b_games[numeric_columns]), 5))

Top Absolute Correlations
users_rated      owned              0.98206
                 num_of_comments    0.97148
num_of_comments  num_of_weights     0.94411
owned            num_of_comments    0.93862
users_rated      num_of_weights     0.88896
dtype: float64


In [12]:
#убираем сильно скоррелированные признаки (корр>0.96)
b_games.drop(['owned', 'num_of_comments'], axis = 1, inplace=True)

In [13]:
#берем топ-10 самых популярных игр среди всех пользователей
df_pop = b_games.sort_values(by = ['average_rating'], ascending=False).loc[:,['boardgame_id', 'title', 'average_rating']].head(10)
df_pop

,boardgame_id,title,average_rating
1781,242705,Aeon Trespass: Odyssey,9.35863
516,251661,Oathsworn: Into the Deepwood,9.13598
954,325494,ISS Vanguard,8.91663
1407,359609,Arkham Horror: The Card Game (Revised Edition),8.78661
1870,240271,Core Space,8.71707
882,295785,Euthia: Torment of Resurrection,8.71608
254,235802,Too Many Bones: Undertow,8.69826
1313,364011,Great Western Trail: Argentina,8.69072
828,169427,Middara: Unintentional Malum – Act 1,8.68499
339,224517,Brass: Birmingham,8.64324


In [14]:
#фильтр нужных столбцов в датасете с оценками
ratings10 = ratings.loc[:,['nickname', 'boardgame_id', 'rating']]

In [15]:
#объединяем взимодействия наших пользователей и наш топ-10
ratings10 = ratings10.set_index(['nickname', 'boardgame_id']).join(df_pop.set_index('boardgame_id'), how='left')
ratings10

rating title  average_rating
nickname    boardgame_id                              
happyjosiah 207830           7.0   NaN             NaN
            432              7.0   NaN             NaN
            68448            8.0   NaN             NaN
            173346           5.0   NaN             NaN
            5                9.0   NaN             NaN
...                          ...   ...             ...
Michiel     22345            7.5   NaN             NaN
            528              8.0   NaN             NaN
            160495           9.5   NaN             NaN
            113924           3.0   NaN             NaN
            27588            6.5   NaN             NaN

[1003471 rows x 3 columns]

In [16]:
#делаем ранжирование оценок
ratings10['cumulative_rank'] = ratings10.groupby(level='nickname').cumcount() + 1
ratings10['cumulative_rank'] = ratings10['cumulative_rank'] / ratings10['average_rating']
ratings10

rating title  average_rating  cumulative_rank
nickname    boardgame_id                                               
happyjosiah 207830           7.0   NaN             NaN              NaN
            432              7.0   NaN             NaN              NaN
            68448            8.0   NaN             NaN              NaN
            173346           5.0   NaN             NaN              NaN
            5                9.0   NaN             NaN              NaN
...                          ...   ...             ...              ...
Michiel     22345            7.5   NaN             NaN              NaN
            528              8.0   NaN             NaN              NaN
            160495           9.5   NaN             NaN              NaN
            113924           3.0   NaN             NaN              NaN
            27588            6.5   NaN             NaN              NaN

[1003471 rows x 4 columns]

In [17]:
#смотрим количество взимодействий каждого пользователя
ratings10['users_item_count'] = ratings10.groupby(level='nickname')['average_rating'].transform(np.size)
ratings10

rating title  average_rating  cumulative_rank  \
nickname    boardgame_id                                                  
happyjosiah 207830           7.0   NaN             NaN              NaN   
            432              7.0   NaN             NaN              NaN   
            68448            8.0   NaN             NaN              NaN   
            173346           5.0   NaN             NaN              NaN   
            5                9.0   NaN             NaN              NaN   
...                          ...   ...             ...              ...   
Michiel     22345            7.5   NaN             NaN              NaN   
            528              8.0   NaN             NaN              NaN   
            160495           9.5   NaN             NaN              NaN   
            113924           3.0   NaN             NaN              NaN   
            27588            6.5   NaN             NaN              NaN   

                          users_item_count  
nickname    boardgame_id                    
happyjosiah 207830                     476  
            432                        476  
            68448                      476  
            173346                     476  
            5                          476  
...                                    ...  
Michiel     22345                      702  
            528                        702  
            160495                     702  
            113924                     702  
            27588                      702  

[1003471 rows x 5 columns]

In [18]:
#считаем метрику map@10 для топ-10
users_count = ratings10.index.get_level_values('nickname').nunique()
map10 = (ratings10["cumulative_rank"] / ratings10["users_item_count"]).sum() / users_count
print(f"MAP@10 = {map10}")

MAP@10 = 0.031380417300205604
